In [1]:
import utils
import torch
from models.basic_model import CDEvaluator
import os
import numpy as np
import pandas as pd
from typing import TypedDict
import xarray as xr
import rioxarray as rxr
import rasterio as rio
from dask.distributed import Client
from dask_cuda import LocalCUDACluster
cluster = LocalCUDACluster(threads_per_worker=4)
client = Client(cluster)
client

2022-06-08 18:16:28,115 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize


Connection method: Cluster object,Cluster type: dask_cuda.LocalCUDACluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 1
Total threads: 4,Total memory: 83.59 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:36943,Workers: 1
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: Just now,Total memory: 83.59 GiB
Comm: tcp://127.0.0.1:42475,Total threads: 4
Dashboard: http://127.0.0.1:39489/status,Memory: 83.59 GiB
Nanny: tcp://127.0.0.1:34431,


In [2]:
args = utils.get_args()
utils.get_device(args)

In [3]:
def load_model():
    args.checkpoint_dir = os.path.join(args.checkpoint_root, args.project_name)
    model = CDEvaluator(args)
    model.load_checkpoint(args.checkpoint_name) 
    model.eval()
    return model
remote_model = client.submit(load_model)
print(remote_model)

<Future: pending, key: load_model-3dc9e6a6d71b4100927f8eb613340e4f>


# Prepare Planet Data

In [5]:
a = rio.vrt.WarpedVRT(rio.open('./catalogs/17_05/mosaic.tif'))
b = rio.vrt.WarpedVRT(rio.open('./catalogs/22_05/mosaic.tif'),transform=a.transform,height=a.height,width=a.width)

ds1 = rxr.open_rasterio(a,chunks=(4,8192,8192),lock=False)
ds2 = rxr.open_rasterio(b,chunks=(4,8192,8192),lock=False)

ds1 = ds1[:3]
ds2 = ds2[:3]

ds1 = ds1/255.0
ds2 = ds2/255.0

m1 = ds1.mean(axis=[1,2])
s1 = ds1.std(axis=[1,2])
m2 = ds2.mean(axis=[1,2])
s2 = ds2.std(axis=[1,2])

# ds = xr.combine_nested([ds1,ds2],concat_dim="time").chunk((2,3,256,256))
ds = xr.combine_nested([ds1,ds2],concat_dim="time")

bands = xr.DataArray([1,2,3],name="band",dims=["band"],coords={"band":[1,2,3]})

first_mu = xr.DataArray(m1.data,name="mean",coords=[bands])
first_std = xr.DataArray(s1.data,name="std",coords=[bands])
second_mu = xr.DataArray(m2.data,name="mean",coords=[bands])
second_std = xr.DataArray(s2.data,name="std",coords=[bands])

mean = xr.concat([first_mu,second_mu],dim="time")
std = xr.concat([first_std,second_std],dim="time")

normalized = (ds-mean)/std
# normalized = normalized.chunk((2,3,256,256))

slices = {}
for coord in ["y","x"]:
    remainder = len(ds.coords[coord])%32
    slice_ = slice(-remainder) if remainder else slice(None)
    slices[coord] = slice_

ds_comb = normalized.isel(**slices)

ds_comb = ds_comb.chunk((2,3,8192,8192))
ds_comb

<xarray.DataArray (time: 2, band: 3, y: 38848, x: 47200)>
dask.array<rechunk-merge, shape=(2, 3, 38848, 47200), dtype=float64, chunksize=(2, 3, 8192, 8192), chunktype=numpy.ndarray>
Coordinates:
  * band         (band) int64 1 2 3
  * x            (x) float64 5.568e+05 5.568e+05 ... 6.984e+05 6.984e+05
  * y            (y) float64 3.415e+06 3.415e+06 ... 3.298e+06 3.298e+06
    spatial_ref  int64 0
Dimensions without coordinates: time

In [26]:
import dask.array
from itertools import product
def predict_chips(data,model)->torch.Tensor:
    result = model._forward_pass(data).cpu().numpy()[0][0]
    return result

def copy_and_predict_chunked(tile,model,token=None):
    out = np.empty(shape=tile.shape[2:], dtype="uint8")
    device = torch.device("cuda")
    w,h = out.shape
    grid = product(range(0, h-h%256, 256), range(0, w-w%256, 256))
    for i,j in grid:
        A = torch.as_tensor(tile[0][np.newaxis,:,j:j+256,i:i+256])
        B = torch.as_tensor(tile[1][np.newaxis,:,j:j+256,i:i+256])
        gpu_chip = {'name':'test',
                    'A':A.float().to(device),
                    'B':B.float().to(device),
                    'L':torch.zeros(1,1,256,256).float().to(device)
               }
        out[j:j+256,i:i+256] = predict_chips(gpu_chip,model)
    return out
    # A = torch.as_tensor(tile[0][np.newaxis, ...])
    # B = torch.as_tensor(tile[1][np.newaxis, ...])
    # gpu_chip = {'name':'test',
    #             'A':A.float().to(device),
    #             'B':B.float().to(device),
    #             'L':torch.zeros(1,1,256,256).float().to(device)
    #            }
    # out = predict_chips(gpu_chip, model)
    # return out

meta = np.array([[]], dtype="uint8")[:0]

predictions_array = ds_comb.data.map_blocks(
    copy_and_predict_chunked,
    meta=meta,
    drop_axis=[0,1],
    model=remote_model,
    name="predict",
)

predictions = xr.DataArray(
    predictions_array,
    coords=ds_comb.drop_vars("band").coords,
    dims=("y", "x"),
)
predictions

<xarray.DataArray 'predict-d0463975986f22b3c3bc991186e87ad2' (y: 38848, x: 47200)>
dask.array<predict, shape=(38848, 47200), dtype=uint8, chunksize=(8192, 8192), chunktype=numpy.ndarray>
Coordinates:
  * x            (x) float64 5.568e+05 5.568e+05 ... 6.984e+05 6.984e+05
  * y            (y) float64 3.415e+06 3.415e+06 ... 3.298e+06 3.298e+06
    spatial_ref  int64 0

In [27]:
predictions[:200,:200].compute()

<xarray.DataArray 'predict-d0463975986f22b3c3bc991186e87ad2' (y: 200, x: 200)>
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)
Coordinates:
  * x            (x) float64 5.568e+05 5.568e+05 ... 5.574e+05 5.574e+05
  * y            (y) float64 3.415e+06 3.415e+06 ... 3.414e+06 3.414e+06
    spatial_ref  int64 0

#### If that works, try the full array

In [28]:
predictions.compute()

<xarray.DataArray 'predict-d0463975986f22b3c3bc991186e87ad2' (y: 38848, x: 47200)>
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)
Coordinates:
  * x            (x) float64 5.568e+05 5.568e+05 ... 6.984e+05 6.984e+05
  * y            (y) float64 3.415e+06 3.415e+06 ... 3.298e+06 3.298e+06
    spatial_ref  int64 0

In [29]:
predictions.rio.to_raster("change_map.tif")

In [ ]:
import matplotlib.colors
from bokeh.models.tools import BoxZoomTool
import panel
import hvplot.xarray

def logo(plot, element):
    plot.state.toolbar.logo = None


zoom = BoxZoomTool(match_aspect=True)
style_kwargs = dict(
    width=450,
    height=400,
    xaxis=False,
    yaxis=False,
)
kwargs = dict(
    x="x",
    y="y",
    rasterize=True,
    cmap='gray',
    aggregator="min",
    colorbar=False,
    tools=["pan", zoom, "wheel_zoom", "reset"],
)

In [ ]:
middle = ds.shape[2] // 2, ds.shape[3] // 2
slice_y = slice(middle[0], middle[0] + 5_000)
slice_x = slice(middle[1], middle[1] + 5_000)

parts = [x.isel(y=slice_y, x=slice_x) for x in [ds, predictions]]

In [ ]:
ds_local, predictions_local = dask.compute(*parts)

In [ ]:
panel.Column(
    panel.Row(
        ds_local.sel(time=0)
        .hvplot.rgb(
            bands="band", rasterize=True, hover=False, title="Austin 05_17", tools=["pan", zoom, "wheel_zoom", "reset"], **style_kwargs
        )
        .opts(default_tools=[], hooks=[logo]),
        predictions_local
        .hvplot.image(title="Changes", **kwargs, **style_kwargs)
        .opts(default_tools=[]),
        ds_local.sel(time=1)
        .hvplot.rgb(
            bands="band",
            rasterize=True,
            hover=False,
            title="Austin 05_22",
            tools=["pan", zoom, "wheel_zoom", "reset"],
            **style_kwargs,
        )
        .opts(default_tools=[], hooks=[logo]),
    ),
)

In [ ]:
from skimage.morphology import (erosion, dilation, closing, opening,
                                area_closing, area_opening)
from skimage.measure import label, regionprops, regionprops_table
from skimage.io import imread, imshow

In [ ]:
knl = np.ones((3,3))
def multi_dil(im, num, element=knl):
    for i in range(num):
        im = dilation(im, element)
    return im
def multi_ero(im, num, element=knl):
    for i in range(num):
        im = erosion(im, element)
    return im

In [ ]:
dilated = multi_dil(predictions_local,7)
closed = area_closing(dilated, 50000)
multi_eroded = multi_ero(closed, 7)
opened = opening(multi_eroded)

In [ ]:
label_im = label(opened)
regions = regionprops(label_im)

df = pd.DataFrame(regionprops_table(label_im,properties=['centroid','area']))

df = df.sort_values(by='area',ascending=False)
df.head()

In [ ]:
ims_17 = []
ims_22 = []
changes = []
for index,row in df.iterrows():
    ims_17.append(ds_local[0,:3,int(round(row['centroid-0']))-128:int(round(row['centroid-0']))+128,int(round(row['centroid-1']))-128:int(round(row['centroid-1']))+128])
    ims_22.append(ds_local[1,:3,int(round(row['centroid-0']))-128:int(round(row['centroid-0']))+128,int(round(row['centroid-1']))-128:int(round(row['centroid-1']))+128])
    changes.append(predictions_local[int(round(row['centroid-0']))-128:int(round(row['centroid-0']))+128,int(round(row['centroid-1']))-128:int(round(row['centroid-1']))+128])

In [ ]:
import matplotlib.pyplot as plt
f,(ax1,ax2,ax3) = plt.subplots(1,3,figsize=(14,4))
ims_17[2].plot.imshow(ax=ax1)
ax1.title.set_text('2017: Undeveloped')
changes[2].plot.imshow(ax=ax2,add_colorbar=False,cmap='gray')
ax2.title.set_text('Change map')
ims_22[2].plot.imshow(ax=ax3)
ax3.title.set_text('2022: Developing residential')
ax1.axis('off')
ax2.axis('off')
ax3.axis('off')
plt.rcParams.update({'font.size': 14})

In [ ]:
f,(ax1,ax2,ax3) = plt.subplots(1,3,figsize=(14,4))
ims_17[5].plot.imshow(ax=ax1)
ax1.title.set_text('2017: Undeveloped')
changes[5].plot.imshow(ax=ax2,add_colorbar=False,cmap='gray')
ax2.title.set_text('Change map')
ims_22[5].plot.imshow(ax=ax3)
ax3.title.set_text('2022: Developing residential')
ax1.axis('off')
ax2.axis('off')
ax3.axis('off')
plt.rcParams.update({'font.size': 14})

In [ ]:
f,(ax1,ax2,ax3) = plt.subplots(1,3,figsize=(14,4))
ims_17[7].plot.imshow(ax=ax1)
ax1.title.set_text('2017: Undeveloped')
changes[7].plot.imshow(ax=ax2,add_colorbar=False,cmap='gray')
ax2.title.set_text('Change map')
ims_22[7].plot.imshow(ax=ax3)
ax3.title.set_text('2022: Developing residential')
ax1.axis('off')
ax2.axis('off')
ax3.axis('off')
plt.rcParams.update({'font.size': 14})

### Store out results to work on classifier

In [ ]:
import pickle
results = {}
results['ims_2017']=ims_17
results['ims_2022']=ims_22
results['change_map'] = changes
pickle.dump(results,open('middle_results.pickle','wb'))

In [ ]:
pred_dataset = predictions.to_dataset()
pred_dataset

In [ ]:
pred_dataset.to_zarr('./change_map.zarr')